In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pickle

def scrape_employee_data(ticker):
    # Define the URL of the webpage you want to scrape
    url = f"https://www.macrotrends.net/stocks/charts/{ticker}/{ticker.lower()}/number-of-employees"

    # Send a request to the webpage
    response = requests.get(url)

    # Parse the HTML content of the webpage using BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")

    # Find the table element that contains the historical employee count data
    table = soup.find("table", {"class": "historical_data_table table"})

    column_names = ["Year", "Number of Employees"]
    # Get the table rows
    rows = []
    if table:
        for tr in table.find_all("tr")[1:]:
            row = []
            for td in tr.find_all("td"):
                row.append(td.text.strip())
            rows.append(row)
    # Store the data in a Pandas dataframe
    df = pd.DataFrame(rows, columns=column_names)
    # df["Number of Employees"] = pd.to_numeric(df["Number of Employees"].replace(",", ""), errors="coerce")
    df["Year"] = pd.to_numeric(df["Year"], errors="coerce")
    df["Number of Employees"] = df["Number of Employees"].str.replace(',', '')
    df["Number of Employees"] = pd.to_numeric(df["Number of Employees"], errors="coerce")
    # Print the dataframe
    return df

In [10]:
layoff_companies = pd.read_csv("layoff_tech_companies.csv")
layoff_companies

,Unnamed: 0,period,revenuePerShare,netIncomePerShare,operatingCashFlowPerShare,freeCashFlowPerShare,cashPerShare,bookValuePerShare,tangibleBookValuePerShare,shareholdersEquityPerShare,...,Funds_Raised,Stage,Date_Added,Country,Percentage,List_of_Employees_Laid_Off,Month,Year,Ticker,Index
0,4,FY,32.733348,3.849057,1.781567,1.249147,7.935144,14.325357,11.148507,14.325357,...,NaN,Post-IPO,2023-03-23 16:47:11,Switzerland,NaN,Unknown,2023-03,2023.0,LOGI,4
1,10,FY,5.599994,-0.528370,1.028252,0.793428,0.769943,19.705087,-3.077890,19.705087,...,NaN,Post-IPO,2023-03-21 23:29:14,United States,0.04,Unknown,2023-03,2023.0,MRVL,10
2,16,FY,74.467678,2.246732,0.021957,0.017731,26.450163,14.571844,-38.736979,14.571844,...,3300.0,Post-IPO,2023-03-22 23:59:38,United States,NaN,Unknown,2023-03,2023.0,EXPE,16
3,20,FY,50.444892,-0.267151,4.588478,-1.657964,6.872706,14.333399,12.342232,14.333399,...,108.0,Post-IPO,2023-03-20 15:07:51,United States,NaN,Unknown,2023-03,2023.0,AMZN,20
4,25,FY,31.844303,2.254025,3.968584,0.950543,15.289061,42.085799,23.639859,42.085799,...,NaN,Post-IPO,2023-03-17 04:11:53,United States,NaN,Unknown,2023-03,2023.0,COHR,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,1642,FY,21.610733,3.551107,5.400341,4.626917,8.091993,18.506814,7.615843,18.506814,...,216.0,Post-IPO,2022-05-26 03:02:05,United States,NaN,Unknown,2022-05,2022.0,PYPL,1642
138,1667,FY,72.150900,-0.760135,-14.605856,-17.742117,2.269144,1.722973,2.882883,1.722973,...,1600.0,Post-IPO,2022-05-10 15:35:42,United States,0.12,Unknown,2022-05,2022.0,CVNA,1667
139,1672,FY,43.414415,-5.057034,-7.753543,-8.140928,15.438220,12.477836,9.927934,12.477836,...,1300.0,Post-IPO,2022-05-10 02:49:25,United States,0.14,Unknown,2022-05,2022.0,VRM,1672
140,1733,FY,16.417323,-2.842520,-7.488189,-7.503937,9.125984,6.440945,6.440945,6.440945,...,1600.0,Post-IPO,2022-05-14 01:20:14,United States,0.15,Unknown,2022-03,2022.0,WISH,1733


In [11]:
layoff_tickers = layoff_companies["Ticker"].tolist()

In [6]:
headcount_dict["LOGI"]

,Year,Number of Employees
0,2022,NaN
1,2021,NaN
2,2020,NaN
3,2019,NaN
4,2018,NaN
5,2017,NaN
6,2016,NaN
7,2015,NaN
8,2014,NaN
9,2013,NaN


In [4]:
scrape_employee_data("LOGI")

,Year,Number of Employees
0,2022,8200.0
1,2021,9000.0
2,2020,6600.0
3,2019,6600.0
4,2018,6100.0
5,2017,6400.0
6,2016,5900.0
7,2015,6900.0
8,2014,6289.0
9,2013,7700.0


In [12]:
headcount_dict = {}

for ticker in layoff_tickers:
    print(f"{ticker} is being scraped...")
    df = scrape_employee_data(ticker)
    headcount_dict[ticker] = df
    # df.to_csv(f"employee_data/{ticker}_employee_data.csv", index=False)

LOGI is being scraped...
MRVL is being scraped...
EXPE is being scraped...
AMZN is being scraped...
COHR is being scraped...
META is being scraped...
NLTX is being scraped...
TTWO is being scraped...
TEAM is being scraped...
SIRI is being scraped...
ABNB is being scraped...
ACCD is being scraped...
ZS is being scraped...
EA is being scraped...
EB is being scraped...
PLTR is being scraped...
ERIC is being scraped...
CRTO is being scraped...
HPQ is being scraped...
DMRC is being scraped...
DOCU is being scraped...
REAL is being scraped...
SMAR is being scraped...
WIX is being scraped...
DOCN is being scraped...
TWLO is being scraped...
OPRT is being scraped...
GDDY is being scraped...
AFRM is being scraped...
ZM is being scraped...
EBAY is being scraped...
SCWX is being scraped...
DELL is being scraped...
OKTA is being scraped...
ADSK is being scraped...
DM is being scraped...
SPLK is being scraped...
PINS is being scraped...
DKNG is being scraped...
FREQ is being scraped...
MTCH is bein

In [6]:
non_layoff_companies = pd.read_csv("non_layoff_stats.csv")

In [7]:
non_layoff_tickers = non_layoff_companies["Ticker"].tolist()

In [8]:
non_layoff_dict = {}

for ticker in non_layoff_tickers:
    if ticker not in non_layoff_dict:
        print(f"{ticker} is being scraped...")
        df = scrape_employee_data(ticker)
        non_layoff_dict[ticker] = df

PLAB is being scraped...
UCLE is being scraped...
EVSBY is being scraped...
TTMI is being scraped...
EPAY is being scraped...
ZTS is being scraped...
RPD is being scraped...
SEDG is being scraped...
TCX is being scraped...
INRD is being scraped...
DNB is being scraped...
NPTN is being scraped...
NTDTY is being scraped...
AACAY is being scraped...
AOSL is being scraped...
SPI is being scraped...
ESTC is being scraped...
ZBRA is being scraped...
GVP is being scraped...
PCTI is being scraped...
AIRG is being scraped...
IPGP is being scraped...
VSH is being scraped...
CIEN is being scraped...
BL is being scraped...
CRGE is being scraped...
ADTTF is being scraped...
AEHR is being scraped...
DQ is being scraped...
CPQQ is being scraped...
NRILY is being scraped...
SNCR is being scraped...
ZVTK is being scraped...
SPDC is being scraped...
ALPP is being scraped...
AKOM is being scraped...
EMKR is being scraped...
KULR is being scraped...
KARO is being scraped...
VLDR is being scraped...
INTV i

In [15]:
# save headcount_dict and non_layoff_dict as pickle files
with open("layoff_headcounts.pickle", "wb") as f:
    pickle.dump(headcount_dict, f)

with open("non_layoff_headcounts.pickle", "wb") as f:
    pickle.dump(non_layoff_dict, f)

In [53]:
scraped_data["Number of Employees"] = pd.to_numeric(scraped_data["Number of Employees"].replace(",", ""), errors="coerce")

In [37]:
scrape_employee_data('BMBL')

,Year,Number of Employees
0,2022,950
1,2021,900
2,2020,700
3,2019,650
4,2018,


In [35]:
scrape_employee_data('ZUO')

,Year,Number of Employees
0,2023,"1,549"
1,2022,"1,393"
2,2021,"1,190"
3,2020,"1,249"
4,2019,"1,221"
5,2018,933
6,2017,
7,2016,


In [55]:
df["Number of Employees"]

0     164000
1     154000
2     147000
3     137000
4     132000
5     123000
6     116000
7     110000
8      97000
9      84400
10     76100
11     63300
12     49400
13     36800
Name: Number of Employees, dtype: int32

In [2]:
print(rows)

[['2022', '164,000'], ['2021', '154,000'], ['2020', '147,000'], ['2019', '137,000'], ['2018', '132,000'], ['2017', '123,000'], ['2016', '116,000'], ['2015', '110,000'], ['2014', '97,000'], ['2013', '84,400'], ['2012', '76,100'], ['2011', '63,300'], ['2010', '49,400'], ['2009', '36,800']]


In [12]:
layoff_tickers = ['SAP', 'OKTA', 'TDOC', 'EXPE', 'W', 'GRPN', 'PLTR', 'MTCH', 'ARVL', 'ZM']
non_layoff_tickers = ['BMBL', 'DV', 'CCUR', 'TTMI', 'NOK', 'UEPS', 'ZUO', 'TOGL', 'CAMT', 'SATS']

In [50]:
layoff_dict = {}
non_layoff_dict = {}
for ticker in layoff_tickers:
    layoff_dict[ticker] = scrape_employee_data(ticker)
for ticker in non_layoff_tickers:
    non_layoff_dict[ticker] = scrape_employee_data(ticker)


In [51]:
layoff_dict

{'SAP':     Year Number of Employees
 0   2022             111,961
 1   2021             107,415
 2   2020             102,430
 3   2019             100,330
 4   2018              96,498
 5   2017              88,543
 6   2016              84,183
 7   2015              76,986
 8   2014              74,406
 9   2013              66,572
 10  2012              64,422
 11  2011              54,346
 12  2010              53,513
 13  2009              47,500,
 'OKTA':    Year Number of Employees
 0  2023               6,013
 1  2022               5,030
 2  2021               2,806
 3  2020               2,248
 4  2019               1,561
 5  2018                    
 6  2017                    
 7  2016                    
 8  2015                    ,
 'TDOC':    Year Number of Employees
 0  2022               5,600
 1  2021               5,100
 2  2020               4,400
 3  2019               2,400
 4  2018               2,000
 5  2017               1,231
 6  2016                 670
 7 

In [43]:
pd.DataFrame(layoff_dict)

ValueError: If using all scalar values, you must pass an index